# IS424: Data Mining & Biz Analytics
### Team: G3T3
### Project: Predicting Loan Default based on Customer Profile
### Model selection: <font color='#0041C2'>Neural Network</font>
---

# 1. Setting up the notebook

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

from sklearn.metrics import recall_score, fbeta_score, roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV

import tensorflow as tf
tf.autograph.set_verbosity(0)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Recall, AUC, Precision
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

tf.get_logger().setLevel("INFO")

In [ ]:
df_train = pd.read_csv("../dataset/train.csv")

y_train = df_train[["risk_flag"]]
x_train = df_train.drop("risk_flag", axis=1)

In [3]:
def target_encoding(df_x, df_y):
    x = df_x.copy()
    
    # Target Encoding — categorical columns with high cardinality: profession, city, state
    profession_target_enc = TargetEncoder()
    x["profession_encoded"] = profession_target_enc.fit_transform(x["profession"], df_y)
    
    city_target_enc = TargetEncoder()
    x["city_encoded"] = city_target_enc.fit_transform(x["city"], df_y)
    
    state_target_enc = TargetEncoder()
    x["state_encoded"] = state_target_enc.fit_transform(x["state"], df_y)
    
    x.drop("profession", axis=1, inplace=True)
    x.drop("city", axis=1, inplace=True)
    x.drop("state", axis=1, inplace=True)
    return x

scale_features = ['income','age','experience']
x_train = target_encoding(x_train, y_train)

scaler = MinMaxScaler()
x_train[scale_features] = scaler.fit_transform(x_train[scale_features])

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


# 2. Running base model

In [4]:
# https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
# https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2
skf = StratifiedKFold(shuffle=True, n_splits=3, random_state=2021)

def create_run_model(lr, bs, x_train, y_train, x_val, y_val):
    model = Sequential()
    model.add(Dense(120, input_shape=(9,), activation="relu"))
    model.add(Dense(120, activation="relu"))
    model.add(Dense(120, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(learning_rate=lr), 
                  metrics=["accuracy", Recall(), tfa.metrics.FBetaScore(num_classes=2, beta=2.0), AUC()])

    y_train = to_categorical(y_train, 2)
    y_val = to_categorical(y_val, 2)
    
    es = EarlyStopping(monitor="loss", patience=5, min_delta=0.001)
    model_fit = model.fit(x_train, y_train, epochs=5, batch_size=bs, verbose=0, callbacks=[es])
    result = model.evaluate(x_val, y_val, verbose=0)
    
    return result

# https://stats.stackexchange.com/questions/164876/what-is-the-trade-off-between-batch-size-and-number-of-iterations-to-train-a-neu
# https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6
# https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
# https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594

learning_rates = [0.01, 0.001]
batch_sizes = [32, 64, 128]
scores = []
variations = []

for train, val in skf.split(x_train, y_train):
    fold_scores = []
    for lr in learning_rates:
        for bs in batch_sizes:
            result = create_run_model(lr, bs, x_train.iloc[train], y_train.iloc[train], x_train.iloc[val], y_train.iloc[val])
            fold_scores.append(result)
            variation = "adam(" + str(lr) + ") bs=" + str(bs)
            if variation not in variations:
                variations.append(variation)
    scores.append(fold_scores)
    
# fold 1 - adam(0.01) 32 | adam(0.01) 64 | adam(0.01) 128 | adam(0.001) 32 | adam(0.001) 64 | adam(0.001) 128
# fold 2 - adam(0.01) 32 | adam(0.01) 64 | adam(0.01) 128 | adam(0.001) 32 | adam(0.001) 64 | adam(0.001) 128
# fold 3 - adam(0.01) 32 | adam(0.01) 64 | adam(0.01) 128 | adam(0.001) 32 | adam(0.001) 64 | adam(0.001) 128

2021-11-07 06:16:36.190499: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 06:16:36.292594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 06:16:36.293295: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-07 06:16:36.295085: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
print(scores)

[[[0.37159276008605957, 0.8770089149475098, 0.8770089149475098, array([0.97271734, 0.        ], dtype=float32), 0.8861415982246399], [0.37099289894104004, 0.8770089149475098, 0.8770089149475098, array([0.97271734, 0.        ], dtype=float32), 0.8870834112167358], [0.37097540497779846, 0.8770089149475098, 0.8770089149475098, array([0.97271734, 0.        ], dtype=float32), 0.8881771564483643], [0.3688647449016571, 0.8770089149475098, 0.8770089149475098, array([0.97271734, 0.        ], dtype=float32), 0.893013060092926], [0.3712023198604584, 0.8770089149475098, 0.8770089149475098, array([0.97271734, 0.        ], dtype=float32), 0.8920329809188843], [0.36946383118629456, 0.8770089149475098, 0.8770089149475098, array([0.97271734, 0.        ], dtype=float32), 0.8914708495140076]], [[0.37115901708602905, 0.8769940733909607, 0.8769940733909607, array([0.9727137, 0.       ], dtype=float32), 0.8887231349945068], [0.3718905448913574, 0.8769940733909607, 0.8769940733909607, array([0.9727137, 0.   

In [6]:
loss = {}
recall = {}
fbeta_2 = {}
auc = {}

fold_no = 1

for fold in scores:
    temp_l = []
    temp_r = []
    temp_f = []
    temp_a = []
    
    for variation in fold:
        temp_l.append(variation[0])
        temp_r.append(variation[2])
        temp_f.append(variation[3][0])
        temp_a.append(variation[4])

    loss["Fold " + str(fold_no)] = temp_l
    recall["Fold " + str(fold_no)] = temp_r
    fbeta_2["Fold " + str(fold_no)] = temp_f
    auc["Fold " + str(fold_no)] = temp_a
    
    fold_no += 1

print(recall)
print(variations)    
loss["Average"] = []
recall["Average"] = []
fbeta_2["Average"] = []
auc["Average"] = []

for i in range(len(variations)):
    loss["Average"].append((loss["Fold 1"][i] + loss["Fold 2"][i] + loss["Fold 3"][i]) / 3)
    recall["Average"].append((recall["Fold 1"][i] + recall["Fold 2"][i] + recall["Fold 3"][i]) / 3)
    fbeta_2["Average"].append((fbeta_2["Fold 1"][i] + fbeta_2["Fold 2"][i] + fbeta_2["Fold 3"][i]) / 3)
    auc["Average"].append((auc["Fold 1"][i] + auc["Fold 2"][i] + auc["Fold 3"][i]) / 3)
    
print()
print("===RECALL===")
score_df = pd.DataFrame(data=recall, index=variations)
display(score_df)

print("===FBETA===")
score_df = pd.DataFrame(data=fbeta_2, index=variations)
display(score_df)

print("===AUC===")
score_df = pd.DataFrame(data=auc, index=variations)
display(score_df)

{'Fold 1': [0.8770089149475098, 0.8770089149475098, 0.8770089149475098, 0.8770089149475098, 0.8770089149475098, 0.8770089149475098], 'Fold 2': [0.8769940733909607, 0.8769940733909607, 0.8769940733909607, 0.8769940733909607, 0.8769940733909607, 0.8769940733909607], 'Fold 3': [0.8769940733909607, 0.8769940733909607, 0.8769940733909607, 0.8769791722297668, 0.8769940733909607, 0.8769940733909607]}
['adam(0.01) bs=32', 'adam(0.01) bs=64', 'adam(0.01) bs=128', 'adam(0.001) bs=32', 'adam(0.001) bs=64', 'adam(0.001) bs=128']

===RECALL===


,Fold 1,Fold 2,Fold 3,Average
adam(0.01) bs=32,0.877009,0.876994,0.876994,0.876999
adam(0.01) bs=64,0.877009,0.876994,0.876994,0.876999
adam(0.01) bs=128,0.877009,0.876994,0.876994,0.876999
adam(0.001) bs=32,0.877009,0.876994,0.876979,0.876994
adam(0.001) bs=64,0.877009,0.876994,0.876994,0.876999
adam(0.001) bs=128,0.877009,0.876994,0.876994,0.876999


===FBETA===


,Fold 1,Fold 2,Fold 3,Average
adam(0.01) bs=32,0.972717,0.972714,0.972714,0.972715
adam(0.01) bs=64,0.972717,0.972714,0.972714,0.972715
adam(0.01) bs=128,0.972717,0.972714,0.972714,0.972715
adam(0.001) bs=32,0.972717,0.972714,0.972700,0.972710
adam(0.001) bs=64,0.972717,0.972714,0.972714,0.972715
adam(0.001) bs=128,0.972717,0.972714,0.972714,0.972715


===AUC===


,Fold 1,Fold 2,Fold 3,Average
adam(0.01) bs=32,0.886142,0.888723,0.884765,0.886543
adam(0.01) bs=64,0.887083,0.885814,0.885813,0.886237
adam(0.01) bs=128,0.888177,0.889880,0.888356,0.888804
adam(0.001) bs=32,0.893013,0.891529,0.890748,0.891763
adam(0.001) bs=64,0.892033,0.892850,0.892446,0.892443
adam(0.001) bs=128,0.891471,0.892459,0.892213,0.892048
